In [1]:
import pandas as pd
import numpy as np
import keras
import csv
from keras.models import Sequential
from keras.layers.recurrent import LSTM
from keras.layers import Dense, Dropout, Conv1D, MaxPooling1D

Using TensorFlow backend.
C:\Users\zippy\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\zippy\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\zippy\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\zippy\anaconda3\envs\te

In [2]:
# This function should transform data into a usable format 
def featureProcess(path):
    data = pd.read_csv(path)
    #get audio data from vect1 to vect148
    featureData = data.loc[:, "vect_1":"vect_148"]
    
    #get feature data from loudness to vect148
    #featureData = data.loc[:, "loudness":"vect_148"]
    
    # normalization
    featureData=(featureData-featureData.mean())/featureData.std()
    
    return featureData.values.tolist()

In [3]:
# This function should transform data into a usable format 
def labelProcess(path):
    data = pd.read_csv(path)
    #get audio data from vect1 to vect148
    labels = data.loc[:,"genre"].values.tolist()
    
    
    return labels

In [4]:
def toOneHot(classes, labels):
    one_hot_labels = []

    for label in labels:
        one_hot_vector = [0] * len(classes)
        one_hot_vector[classes.index(label)] = 1

        one_hot_labels.append(one_hot_vector)
    return one_hot_labels

In [5]:
def LSTMModel(trainData, trainLabel):
    
    num_classes = len(trainLabel[0])  # total classes to train
    #audio feature
    model.add(LSTM(50,  dropout=0.25, recurrent_dropout=0.25, return_sequences=False, input_shape=(148, 1), kernel_initializer="random_uniform"))
    
    #all feature
    #model.add(LSTM(50,  dropout=0.25, recurrent_dropout=0.25, return_sequences=False, input_shape=(154, 1), kernel_initializer="random_uniform"))
    
    model.add(Dense(num_classes, activation="softmax", kernel_initializer="random_uniform"))
    
    model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])
    return model

In [6]:
def train(trainData, trainLabel, model):
    
    batch_size = 32  #the number of data to feed into model per batch
    num_epoch = 100  # go through your training data epoch times
    
    #callbacks stop training if val_loss is not improving.
    callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose = 1)]
    
    history = model.fit(trainData, trainLabel, validation_split=0.2, epochs = num_epoch, batch_size=batch_size, verbose=1, callbacks = callbacks)
                 
    return history, model

In [7]:
def evaluate(validData, validLabel, model):
    loss, accuracy = model.evaluate(validData, validLabel)
    return loss, accuracy

In [8]:
def predict(testData, model):
    result = model.predict_classes(testData, verbose=1)
    return result

In [9]:
trainData = np.expand_dims(np.array(featureProcess(r"D:\Unimelb\Introduction to Machine Learning\assignment2\dataset\dataset\train_features.csv")), axis=2)
validData = np.expand_dims(np.array(featureProcess(r"D:\Unimelb\Introduction to Machine Learning\assignment2\dataset\dataset\valid_features.csv")), axis=2)
testData = np.expand_dims(np.array(featureProcess(r"D:\Unimelb\Introduction to Machine Learning\assignment2\dataset\dataset\test_features.csv")), axis=2)

In [10]:
trainLabel = labelProcess(r"D:\Unimelb\Introduction to Machine Learning\assignment2\dataset\dataset\train_labels.csv")
validLabel = labelProcess(r"D:\Unimelb\Introduction to Machine Learning\assignment2\dataset\dataset\valid_labels.csv")
#get classes
classes = list(set(trainLabel))

oneHotTrain = np.array(toOneHot(classes, trainLabel))
oneHotvalid = np.array(toOneHot(classes, validLabel))

In [11]:
model = Sequential()
model = LSTMModel(trainData, oneHotTrain)
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 50)                10400     
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 408       
Total params: 10,808
Trainable params: 10,808
Non-trainable params: 0
_________________________________________________________________


In [12]:
history = train(trainData, oneHotTrain, model)
print("Finish training")

Instructions for updating:
Use tf.cast instead.
Train on 6142 samples, validate on 1536 samples
Epoch 1/100
6142/6142 [==============================] - 60s 10ms/step - loss: 1.9535 - acc: 0.2318 - val_loss: 1.8952 - val_acc: 0.2435
Epoch 2/100
6142/6142 [==============================] - 57s 9ms/step - loss: 1.8913 - acc: 0.2548 - val_loss: 1.8388 - val_acc: 0.3451
Epoch 3/100
6142/6142 [==============================] - 56s 9ms/step - loss: 1.8536 - acc: 0.2906 - val_loss: 1.8323 - val_acc: 0.3105
Epoch 4/100
6142/6142 [==============================] - 54s 9ms/step - loss: 1.7907 - acc: 0.3269 - val_loss: 1.7653 - val_acc: 0.3477
Epoch 5/100
6142/6142 [==============================] - 53s 9ms/step - loss: 1.7338 - acc: 0.3486 - val_loss: 1.6472 - val_acc: 0.4089
Epoch 6/100
6142/6142 [==============================] - 54s 9ms/step - loss: 1.7115 - acc: 0.3634 - val_loss: 1.6449 - val_acc: 0.4062
Epoch 7/100
6142/6142 [==============================] - 53s 9ms/step - loss: 1.6990 - 

In [13]:
loss, accuracy = evaluate(validData, oneHotvalid, model)
print("loss, accuracy:", loss, accuracy)

450/450 [==============================] - 1s 3ms/step
loss, accuracy: 1.7296871863471137 0.37555555555555553


Predict

In [14]:
trackID = pd.read_csv(r"D:\Unimelb\Introduction to Machine Learning\assignment2\dataset\dataset\test_features.csv").loc[:, "trackID"].values.tolist()
prediction = predict(testData, model)

428/428 [==============================] - 2s 4ms/step


In [15]:
results_name = []

for result in prediction:
    results_name.append(classes[result])
    
result = list(zip(trackID,results_name))

In [17]:
file = "audio_LSTM_outPut.csv"
#file = "meta+audio_LSTM_outPut.csv"

with open(file, 'w', newline='') as csvFile:
    writer = csv.writer(csvFile)
    writer.writerow(["trackID", "genre"])
    writer.writerows(result)

all feature
1. 33.56 
2. 35.56
3. 35.33
4. 32.89
5. 32.22
6. 34.67
7. 35.33
8. 32.89
9. 34.67
10. 34

MFCCs
1. 35.78
2. 39.56
3. 34.89
4. 34.89
5. 36
6. 35.78
7. 35.78
8. 37.22
9. 34.78
10. 36